In [1]:
# first of all import application of VGG16 from keras which image size is 224x224
from keras.applications import VGG16
image_rows = 224
image_cols = 224
# now load the VGG16 model  from imagenet 
vgg16 = VGG16(weights = 'imagenet',include_top = False, input_shape = (image_rows,
                                                                      image_cols,3))

58889256/58889256 [==============================] - 0s 0us/step


In [2]:
# now let's print our layers
for(i,layer) in enumerate (vgg16.layers):
    print (str(i) + " "+ layer.__class__.__name__,layer.trainable)

0 InputLayer True
1 Conv2D True
2 Conv2D True
3 MaxPooling2D True
4 Conv2D True
5 Conv2D True
6 MaxPooling2D True
7 Conv2D True
8 Conv2D True
9 Conv2D True
10 MaxPooling2D True
11 Conv2D True
12 Conv2D True
13 Conv2D True
14 MaxPooling2D True
15 Conv2D True
16 Conv2D True
17 Conv2D True
18 MaxPooling2D True


In [3]:
# first of all import application of VGG16 from keras which image size is 224x224
from keras.applications import VGG16
image_rows = 224
image_cols = 224
# now Reload the VGG16 model  from imagenet  without the top or fully connected layer 
vgg16 = VGG16(weights = 'imagenet',include_top = False, input_shape = (image_rows,
                                                                      image_cols,3))
#here we freeze the last four layers
# layers are set to trainable as True by default 
for layer in vgg16.layers :
    layer.trainable = False

#let's print our layers
# now let's print our layers
for(i,layer) in enumerate (vgg16.layers):
    print (str(i) + " "+ layer.__class__.__name__,layer.trainable)

0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


In [4]:
#create the top or head model that will be  placed ontop of the bottom layers

def addTopModel(bottom_model,num_classes,D=256):
    top_model = bottom_model.output 
    top_model = Flatten(name = "flatten")(top_model)
    top_model = Dense(D,activation = "relu")(top_model)
    top_model = Dropout(0.3)(top_model)
    top_model = Dense(num_classes,activation = "softmax")(top_model)
    return top_model
   

In [5]:
from keras.models import Sequential
from keras.layers import Dense,Dropout , Activation ,Flatten
from keras.layers import Conv2D,MaxPooling2D,ZeroPadding2D
#from keras.layers.normalization import BatchNormalization
#from keras.layers.normalization.BatchNormalization
from keras.models import Model

In [6]:
num_classes = 17
FC_Head = addTopModel(vgg16,num_classes)
model = Model(inputs = vgg16.input,outputs = FC_Head)
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [7]:
from keras.preprocessing.image import ImageDataGenerator 
train_data_dir = '../input/flower-dataset/flowers/train'
validation_data_dir = '../input/flower-dataset/flowers/validation'

In [8]:
train_datagen = ImageDataGenerator(rescale = 1/255,rotation_range = 20,
                                  width_shift_range = 0.2,height_shift_range =0.2,
                                  horizontal_flip = True,fill_mode ='nearest')

In [9]:
validation_datagen = ImageDataGenerator(rescale = 1/255)

In [10]:
#change tha batch size according to your system RAM
train_batchsize = 16
val_batchsize =10

In [11]:
train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                target_size = (image_rows,image_cols))

Found 1190 images belonging to 17 classes.


In [12]:
validation_generator = validation_datagen.flow_from_directory(validation_data_dir,
                                                             target_size = (image_rows,image_cols),
                                                             batch_size = val_batchsize,
                                                             class_mode = 'categorical',
                                                              shuffle = False 
                                                             )

Found 170 images belonging to 17 classes.


In [13]:
#training our top layers 
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint,EarlyStopping
checkpoint = ModelCheckpoint ("Trained MOdels/flowers_vgg.h5",
                             monitor = "val_loss", mode = "min",
                             save_best_only = True,
                             verbose = 1)
earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)


#now use call backs into a call list 
callbacks = [earlystop ,checkpoint]
# now compile the model
model.compile(loss = 'categorical_crossentropy',
             optimizer = RMSprop(lr = 0.001) ,metrics = ['accuracy'])


/opt/conda/lib/python3.7/site-packages/keras/optimizers/optimizer_v2/rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [14]:
nb_train_sample = 1190
nb_validadtion_sample = 170
epochs = 3
batch_size = 64 
history = model.fit_generator(train_generator,steps_per_epoch=nb_train_sample//batch_size,
                              epochs = epochs, callbacks= callbacks,
                              validation_data= validation_generator,
                              validation_steps= nb_validadtion_sample//batch_size)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/3
18/18 [==============================] - ETA: 0s - loss: 7.3113 - accuracy: 0.1400
Epoch 1: val_loss improved from inf to 2.36864, saving model to Trained MOdels/flowers_vgg.h5
18/18 [==============================] - 29s 877ms/step - loss: 7.3113 - accuracy: 0.1400 - val_loss: 2.3686 - val_accuracy: 0.2500
Epoch 2/3
18/18 [==============================] - ETA: 0s - loss: 2.2970 - accuracy: 0.2691
Epoch 2: val_loss did not improve from 2.36864
18/18 [==============================] - 11s 625ms/step - loss: 2.2970 - accuracy: 0.2691 - val_loss: 2.6000 - val_accuracy: 0.0500
Epoch 3/3
18/18 [==============================] - ETA: 0s - loss: 2.2943 - accuracy: 0.3145
Epoch 3: val_loss improved from 2.36864 to 1.95152, saving model to Trained MOdels/flowers_vgg.h5
18/18 [==============================] - 11s 595ms/step - loss: 2.2943 - accuracy: 0.3145 - val_loss: 1.9515 - val_accuracy: 0.2500


In [15]:
model.save("trained models/flowers_vgg.h5")

# Now implement  Vgg16 using image size 64x64

In [16]:
# first of all import application of VGG16 from keras which image size is 64x64
from keras.applications import VGG16
image_rows = 64
image_cols = 64
# now Reload the VGG16 model  from imagenet  without the top or fully connected layer 
vgg16 = VGG16(weights = 'imagenet',include_top = False, input_shape = (image_rows,
                                                                      image_cols,3))
#here we freeze the last four layers
# layers are set to trainable as True by default 
for layer in vgg16.layers :
    layer.trainable = False

#let's print our layers
# now let's print our layers
for(i,layer) in enumerate (vgg16.layers):
    print (str(i) + " "+ layer.__class__.__name__,layer.trainable)

0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


# create new model using an image size 64x64

In [17]:
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
#from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = '../input/flower-dataset/flowers/train'
validation_data_dir = '../input/flower-dataset/flowers/validation'

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 16
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(image_rows, image_cols),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(image_rows, image_cols),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

# Re-loads the VGG16 model without the top or FC layers
vgg16 = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (image_rows, image_cols, 3))

# Freeze layers
for layer in vgg16.layers:
    layer.trainable = False
    
# Number of classes in the Flowers-17 dataset
num_classes = 17

FC_Head = addTopModel(vgg16, num_classes)

model = Model(inputs=vgg16.input, outputs=FC_Head)

print(model.summary())

Found 1190 images belonging to 17 classes.
Found 170 images belonging to 17 classes.
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                        

In [18]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
                   
checkpoint = ModelCheckpoint("/home/deeplearningcv/DeepLearningCV/Trained Models/flowers_vgg_64.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 5,
                          verbose = 1,
                          restore_best_weights = True)

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                              factor = 0.2,
                              patience = 3,
                              verbose = 1,
                              min_delta = 0.00001)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint, reduce_lr]

# Note we use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.0001),
              metrics = ['accuracy'])

nb_train_samples = 1190
nb_validation_samples = 170
epochs = 25
batch_size = 32

history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

model.save("/home/deeplearningcv/DeepLearningCV/Trained Models/flowers_vgg_64.h5")

Epoch 1/25


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:41: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


37/37 [==============================] - ETA: 0s - loss: 2.9198 - accuracy: 0.0895
Epoch 1: val_loss improved from inf to 2.52758, saving model to /home/deeplearningcv/DeepLearningCV/Trained Models/flowers_vgg_64.h5
37/37 [==============================] - 7s 161ms/step - loss: 2.9198 - accuracy: 0.0895 - val_loss: 2.5276 - val_accuracy: 0.2600 - lr: 1.0000e-04
Epoch 2/25
37/37 [==============================] - ETA: 0s - loss: 2.6498 - accuracy: 0.1639
Epoch 2: val_loss improved from 2.52758 to 2.45706, saving model to /home/deeplearningcv/DeepLearningCV/Trained Models/flowers_vgg_64.h5
37/37 [==============================] - 5s 131ms/step - loss: 2.6498 - accuracy: 0.1639 - val_loss: 2.4571 - val_accuracy: 0.2200 - lr: 1.0000e-04
Epoch 3/25
37/37 [==============================] - ETA: 0s - loss: 2.4661 - accuracy: 0.2399
Epoch 3: val_loss improved from 2.45706 to 2.23144, saving model to /home/deeplearningcv/DeepLearningCV/Trained Models/flowers_vgg_64.h5
37/37 [===================